In [ ]:
import json
import time
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline
import paho.mqtt.client as mqtt
import pickle


with open("trained_random_forest_model.pkl", "rb") as model_file:
    model = pickle.load(model_file)

class RealTimeProcessor:
    def __init__(self, desired_freq=32, interpolation_freq=50):
        self.desired_freq = desired_freq   
        self.interpolation_freq = interpolation_freq / 1000   
        self.data_buffer = []
        self.filtered_data = []
        self.interpolated_data_list = []

    def select_32_samples(self, data_list):
     
        data_list = sorted(data_list, key=lambda x: x['timestamp'])
        df = pd.DataFrame(data_list)
        df = df.drop_duplicates(subset=['timestamp'])  
        if len(df) >= self.desired_freq:
            indices = np.linspace(0, len(df) - 1, self.desired_freq, dtype=int)
            selected_data = df.iloc[indices].to_dict('records')
        else:
            selected_data = df.to_dict('records')
        return selected_data

    def process_data(self):
        selected_data = self.select_32_samples(self.data_buffer)
        df = pd.DataFrame(selected_data)
        df = df.sort_values(by='timestamp')
        
        self.filtered_data = self.apply_ekf(df)
        self.interpolated_data = self.interpolate_data(self.filtered_data)

        self.predict_data(self.interpolated_data)
        self.data_buffer.clear()

    def apply_ekf(self, df):
        filtered_data = []
        for index, row in df.iterrows():
            z = np.array([row['ax'], row['ay'], row['az'], row['gx'], row['gy'], row['gz']])
            x_est = self.ekf_filter(z)
            filtered_data.append({
                'timestamp': row['timestamp'],
                'ax_filtered': x_est[0],
                'ay_filtered': x_est[1],
                'az_filtered': x_est[2],
                'gx_filtered': x_est[3],
                'gy_filtered': x_est[4],
                'gz_filtered': x_est[5]
            })
        return pd.DataFrame(filtered_data)

    def ekf_filter(self, z):
        x_est = z  
        return x_est

    def interpolate_data(self, df):
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        timestamps_unix = df['timestamp'].astype(np.int64) // 10**6  
    
        values = df[['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered']].values
        new_timestamps_unix = np.linspace(timestamps_unix.min(), timestamps_unix.max(), 50)
    
        try:
            cs = CubicSpline(timestamps_unix, values, axis=0, bc_type='natural')
            new_values = cs(new_timestamps_unix)
        except Exception as e:
            print(f"Interpolasyon hatası: {e}")
            return pd.DataFrame()
    
        interpolated_df = pd.DataFrame(new_values, columns=['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered'])
        interpolated_df['timestamp'] = pd.to_datetime(new_timestamps_unix, unit='ms')
    
        print("\nInterpolasyon Sonucu (50 Hz):")
        print(interpolated_df.head())
    
        self.interpolated_data_list.append(interpolated_df)
    
        return interpolated_df


    def predict_data(self, df):
    
        if len(df) >=50:
            features = df[['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered']].values.flatten().reshape(1, -1)
            try:
                
                prediction = model.predict(features)
                print("Gerçek Zamanlı Tahmin Sonucu:", prediction[0])
                
            except Exception as e:

                print("Tahmin yaparken hata oluştu:", e)
        else:

            print("50 örnek toplanamadı, tahmin yapılmıyor.")
            print(len(df))

    def add_data(self, data):

        self.data_buffer.append(data)
        print(f"Toplanan veri sayısı: {len(self.data_buffer)}")  
        if len(self.data_buffer) >= self.desired_freq:
            self.process_data()
        
        time.sleep(0.03)

mqtt_broker = "192.168.108.135"
mqtt_port = 1883
topic = "esp32_4/data"
processor = RealTimeProcessor()

def on_message(client, userdata, message):
    payload = message.payload.decode('utf-8')
    data = json.loads(payload)
    data['timestamp'] = time.time() * 1000  
    processor.add_data(data)

client = mqtt.Client()
client.on_message = on_message
client.connect(mqtt_broker, mqtt_port, 60)
client.subscribe(topic)
client.loop_start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    client.loop_stop()
    client.disconnect()
